# Chapter 2: Multi Armed Bandits
Let's explore the different ideas of Reinforcement Learning.

## k-armed bandit problem
At each timestamp, we need to choose 1 out of $k$ possible choices. Each option is associated with a reward. This value can be seen as a result of some hidden probability distribution. Let's consider refer to its expected value as the **value** of an action. Assuming $A_t$ is the action taken at timestamp $t$ and $R_t$ the corresponding reward, then 
$$ q_{*}(a) = E[R_t | a = A_t]$$
At each timestamp, we have an estimation of the reward value of a given action, say $Q_t(a)$. There 3 points
1. we would like $Q_t(a)$ to be as close as possible to $q_{*}(a)$
2. As the number of actions is finite, we can know which action has the most immediate expected / esimated reward. 
3. Choosing the action with the largest expected estimated reward is referred to as **exploitation**. Otherwise, it is called **exploration**


## Action-value methods
As mentioned above, among the most important components, aspects of RL is to estimate the rewards of different actions. The most straightforward way is calculating the average of rewards when choosing that exact action: 
$\begin{align} 
Q_t(a) = \frac{\sum_{i=1}^{t - 1} R_i \cdot 1_{A_i = a}}{\sum_{i=1}^{t - 1}  1_{A_i = a}}
\end{align}$
where this is translated as the fraction of the: sum of rewards at time stamps where the action $a$ was taken, over the number of times the action $a$ was taken.   
There are different ways to estimate the rewards at a given timestamp. Nevertheless, we would elaborate on that later.  
The greedy approach is to choose $a$ such that:
$$ A_t = argmax_a Q_t(a)$$ 
A slightly asymptotically better policy is choosing the greedy option only $1 - \epsilon$ of the time, while choosing a random option out of all options in the $\epsilon$ left.   
A computational remark: it is enough to save the number of times / steps each action was chosen, and the sum of rewards up till the current timestamp. Introducing some notation, we have the following:
1. $Q_n = \frac{\sum_{i = 1}^{n - 1} R_i}{n - 1}$, where $R_i$ is the reward at the $i$-th timestamp
2. $Q_{n + 1} = \frac{\sum_{i = 1}^{n} R_i}{n} = \frac{R_n + (n - 1) \cdot \frac{\sum_{i = 1}^{n - 1} R_i}{n-1}}{n}$
3. $Q_{n + 1} = \frac{R_n + Q_n (n - 1)}{n}= Q_n + \frac{R_n - Q_n}{n}$

The last equation provides a framework of the efficient computation of rewards and estimated rewards

In [1]:
# sounds like time to code our problems and have some fun with it
from typing import Union
from _collections_abc import Sequence
import numpy as np
import random
# set the seeds

random.seed(69)
np.random.seed(69)

class kArmBandit:
    def __init__(self, k: int=5, rewards_means: list[Union[float, int]]=None, rewards_variances: Union[float, list[float]]=None) -> None:
        # number of options
        self.k = k
        # the means parameters must be a sequence of floats or integers of size k
        assert rewards_means is None or (isinstance(rewards_means, Sequence) and len(rewards_means) == k)
        # the default values for the mean values are random

        if rewards_means is None:        
            rewards_means = [np.random.uniform(-5, 5) for _ in range(k)]

        if rewards_variances is None:
            rewards_variances = [1 for _ in range(k)]

        if isinstance(rewards_means, float) or isinstance(rewards_variances, int):
            rewards_variances = [rewards_variances for _ in range(k)]

        assert isinstance(rewards_variances, Sequence) and len(rewards_variances) == k

        self.rewards = [lambda: np.random.normal(mean, variance, 1) for mean, variance in zip(rewards_means, rewards_variances)]

    def reward(self, option_number: int):
        assert option_number < self.k, 'The option chosen is not available by the arm bandit'
        return self.rewards[option_number]()

    def reward_and_optimal(self, option_number: int):
        # first extract all the possible rewards
        all_rewards = [r() for r in self.rewards]
        # return the reward chosen and the maximum one
        return self.rewards(option_number), max(all_rewards)
    

In [ ]:
# let's create an agent class for that

class Agent:
    def __init__(self, k: int, epsilon:float=0, start_estimate:float=0, update_estimate: callable=None) -> None:
        self.k = k
        # make sure epsilon is in the range [0, 1]
        assert 0 <= epsilon <= 1
        self.epsilon = epsilon # determines how often does the agent make a random non-greedy choice
        self.estimates = [start_estimate for _ in range(self.k)]
        self.step_sizes = [1 for _ in range(self.k)]

        # the default update estimate value is 1 / step_size
        # this is used for average sampling
        if update_estimate is None:
            update_estimate = lambda step_size: 1 / step_size
        
        self.update_estimate =  update_estimate  
        self.total_reward = 0
    
    def chooose(self):
        p = random.random()
        if p < self.epsilon:
            # return a random choice
            return random.choice(list(range(self.k)))
        else:
            # choose the option with the highest estimate
            return np.argmax(self.estimates)
    
    def update_estimates(self, option_num:int, option_reward:float):
        # make sure to add the reward to the total_reward
        self.total_reward += option_reward
        # make sure to update the estimated value of the corresponding reward
        e = self.estimates[option_num] # writing the entire expression repeatedly is not the best option
        self.estimates[option_num] = (e + self.update_estimate(self.step_sizes[option_num]) * (option_reward - e)) 
        # increment the step size for the corresponding option
        self.step_sizes[option_num] += 1 


In [ ]:
# time to create a different class for experimenting with the different settings of the Agent and the ArmBadint
class Experiment:
    def __init__(self, agent: Agent, arm_bandit: kArmBandit, steps: int=1000, exps:int=100) -> None:
        # first make sure they have they operate on the same number of options
        assert agent.k == arm_bandit.k, "The agent and the armbandit must have the same number of options"
        self.agent = agent
        self.bandit = arm_bandit
        # how many will the agent have to choose an option at each trials
        self.steps = steps
        # the number of trials
        self.exps = exps

    def __run_trial(self):
        # each trial should calculate the
        # 1. the average reward at each step
        # 2. the percentage of the optimal rewards so far
        pass        
        


## Optimistic Initial Values
* One approach to driver the agent to explore more is setting optimistic initial action reward estimates. The main reasoning is that the agent will be disappointed with rewards driving him to explore each of the actions several times. This approach is quite limited: 
1. It only drives exploration in the early steps: The initial values will wash off as the agent continues exploring
2. Any approach based on initial values cannot be generalized to non-stationary problems
3. settings such ***optimistic*** values might not be known before hand.

## Nonstationary Problems
The above-mentioned approach works only for problems where the expected value of an action is constant. Nevertheless, they are far from perfect for non-stationary rewards. Among the possible approaches is using a constant weight to calculate the new term with respect to the previous estimate. In other words:
$$ Q_{n + 1} = Q_n + \alpha \cdot (R_n - Q_n)$$
The latter can be expanded:
$$Q_{n + 1} = (1 - \alpha) ^ n \cdot Q_1 + \sum_{i = 1}^{n} \alpha \cdot (1 - \alpha) ^ {n - i} R_i$$
Meaning that the most recent rewards are more significant in the estimation of the next reward.

## Upper-confidence Bound action selection
Exploration is always needed mainly becuase the estimation of the next actions are always uncertain. The $\epsilon$-greedy approach might force exploration but undescriminately. Such exploration approach is obviously far from optimal. One way to improve our exploration process is to use the UPPER CONFIDENCE BOUND action selection mechanism:
$$ A_t = argmax_a [Q_{t}(a) + c \cdot \sqrt{\frac{\ln(t)}{N_t(a)}}] $$
where the first term is clearly exploitation, while the 2nd is exploration:  

# Chapter 3: Finite Markov Decision Process

Markov Decision Proceeses are formatlization of the RL problem for which, the action and the previous state completely determine the reward as well as the environment's next state. Some math for fun:
$$\begin{align}
p(s',r | s, a)  = Pr(S_t = s, R_t = r | S_{t - 1}, A_{t - 1} = a)
\end{align}
$$
The assumption here is that the sets of rewards, states and actions are finite and therefore this equality holds: 
$$\sum_{r \in R} \sum_{s' \in S} p(s', r | s, a) = 1$$

Certain environments and RL tasks can be broken into repeatable sections / cycles called episodes. For such problems, we are trying to maximize the mathematical expression:
$$ G_t = \sum_{i}^T R_{t + i}$$
Where $T$ is the episode length. The main issue here is that such episodes are not guaranteed to exist. Thus, we introduce a better formulation:

The main characteristics of a periodic task is:
1. The terminal state where the episode cannot proceed further.
2. The different episods are independent and the start state can be one of predefinied states

Nevertheless, not all tasks can be modeled as periodic, as they don't break naturally into periods. Therefore, The objective should be changed accordingly:
$$ G_t = \sum_{k = 0}^{\infin } \gamma ^ k \cdot R_{t + k + 1}$$
Where the $ 0 \leq \gamma < 1$ is simply a discounting factor reflecting the imprtance of long-term rewards. The lower the values, the less far-sighted the model, as the future rewards will soon be practically nullified by the discounting factor.

## Policies and Value Functions
RL agents generally are built upon values functions: mathematical functions that estimate the quality of a given (current) state. In additional to value functions, we have policies: $\pi(a|s)$ which determine the probability of a given action $a$ in a given state $s$. The mind of the agent is literally its policy. Learning can manisfest itself in creating a policy that assigns high probabilities to states with high rewards. The mathematical expression of high reward is basically the expected value of the random variable $G_t$ given a state at any point in time.
$$ v_{\pi}(s) = E_{\pi}[G_t|S_t = s]$$
The value of a terminal state if any is always $0$. Extending the definition a bit further, we can define action-value function: 
$$ q_{\pi}(s, a) = E_{\pi}[G_t|S_t = t, A_t = a]$$
Such expressions are not of much use as they don't offer (at least immediatly) practical means to calculate them. Let's use one simple identity:
$$G_t = R_{t + 1} + \gamma \cdot G_{t + 1}$$
Replacing this expression in the expression of $v_{\pi}$, we obtain:

$\begin{align} 
v_{\pi}(s) &= E_{\pi}[G_t|S_t = s] \\
            &= E_{\pi}[R_{t + 1} + \gamma \cdot G_{t + 1} | S_t = s] \\
            &= \sum_{a} \pi(a | s) \cdot \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot E_{\pi}[G_{t + 1} | S_{t + 1} = s']] \\
v_{\pi}(s)  &= \sum_{a} \pi(a | s) \cdot \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot v_{\pi}(s')]
\end{align}$
This formula known as the Bellman equation gives us a way of computing $v_{\pi}(s)$ in terms of following states $s'$. 

Let's consider the same equation but for state-action pair: $q_{\pi}(s, a) = E_{\pi}[G_t|(S_t = s, A_t = a)]$.
$\begin{align} 
q_{\pi}(s, a) &= E_{\pi}[G_t|(S_t = s, A_t = a)] \\
            &= E_{\pi}[R_{t + 1} + \gamma \cdot G_{t + 1} | S_t = s, A_t = a] \\
            &= \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot E_{\pi}[G_{t + 1} | S_{t + 1} = s']] \\
q_{\pi}(s, a)  &= \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \sum_{a' \in A_t(s')} \pi(a'|s')\cdot q_{\pi}(s', a')]
\end{align}$


Further down the line, we will consider different ways, to compute, estimate $v_{\pi}(s)$ as well as $q_{\pi}(s, a)$   
Solving an RL problems could be seen as maximizing the policy's return on the long run. For a finite MDP, we can define an optimal value policy as follows:   
policy $\pi^{*}$ such as $v_{\pi^{*}}(s) \geq v_{\pi}(s)$ for all states $s$ and all policies $\pi$.   
An intuitive non-formal idea behind the proof of the existence of optimal policy is as follows:
1. Assuming policy $\pi_1$ has a better value function for a set of states $S_1$ while $\pi_2$ has a better value function of a set of states $S_2$.
2. The third policy $\pi_3$ defined as $\pi_1$ for $S_1$ and $\pi_2$ for S_2 is a policy whose value function is larger than all value function for both $\pi$'s 


* Given a policy $\pi$ we can compute the value function of each state by solving a linear system of $|S|$ equations and $|S|$ unknowns. This is not enough to find the optimal policy as the number of possibilities is just untrackable. Nevertheless, the Bellman equations can still help up optimize the process.
* Let's consider the following equations:

$\begin{align} 
v_{\pi}^{*}(s)  &= \sum_{a} \pi(a | s) \cdot \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot v_{\pi}(s')] \\
 &= \sum_{a} \pi^{*}(a | s) \cdot \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot v_{\pi}^{*}(s')]  && \text{The optimal policy should assign 1 to the action with the highest expected reward and 0 otherwise} \\
v_{\pi}^{*}(s) &= \max_{a} \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot v_{\pi}^{*}(s')]
\end{align}$
Let's elaborate more on the last transition. The expression $S = \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot v_{\pi}^{*}(s')]$ depends on $s', s, a, r$. A policy controls the distribution of $a$ given $s$. The choice of $a$ determines the possible values of $r$ and $s'$ and therefore, determine the value of $S$. We can simply find the action $a$ that maximizes $S$ assign it a probability $1$ and $0$ for the other actions assuring optimility.


Having the optimal state functions, we can easily find the optimal policy: For each state $s$, find all actions $a$ for which $\sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot v_{\pi}^{*}(s')]$ is maximal. In other words,
Even Better, $$\pi^{*}(s) = argmax_{a}(\sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot v_{\pi}^{*}(s')])$$
$$\pi^{*}(s) = argmax_{a}(q^{*}(a,s))$$

We can also write $q^{*}(s,a)$ in terms of $v^{*}(s)$ as follows:
$$ q^{*}(s,a) = \sum_{s' \in S(a), r \in R} p(s', r |a, s) \cdot [r + \gamma \cdot v^{*}(s')]$$

# Chpater 4: DP for Policy Evaluation and Control
* policy evaluation is the task of finding the value function $V_{\pi}$ given a policy $\pi$. In other words, find $V_{\pi}(s)$ for every state $s$ in the states' space.
* policy control: is the task of improving an existing policy $\pi$.
* Both these tasks can be done if we have access to the dynamics function $p$ 

## Policy Evaluation
Let's recall the Bellman equation for a state function:
$$v_{\pi}(s) = \sum_{a} \pi(a | s) \cdot \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot v_{\pi}(s')]$$
Simply we can consider $$V_{k + 1}(s) = \sum_{a} \pi(a | s) \cdot \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot v_{k}(s')]$$
for every single state $s$.
The algorithm is pretty simple:
1. randomly initialize two arrays representing the states: $V_{old}$, $V_{new}$
2. define the difference between the $2$ arrays
3. update each state in $V_old$, when all states are updated copy $V_{new}$ to $V_{old}$
4. stop when the difference between 

## Policy Iteration
Policy control: The task of improving a given policy. The main idea is to use the Bellman optimality equation on a given policy. So given $\pi$, we will have $\pi^{'}$ a new greedy policy that chooses the action with the highest value function accroding to $\pi$
$$\pi^{'}(s) = argmax_a \sum_{a} \pi(a | s) \cdot \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot v_{\pi}(s')]$$ 
According to the policy improvement theorem, $\pi^{'}$ is better than $\pi$. In fact strictly better, unless $\pi$ is already optimal.   
Using policy control and policy we can build a general framework: policy iteration as follows:

1. Random initialization of $\pi$ and $v_\pi$
2. Policy Evaluation
3. $\pi_g$ = Policy Improvement($\pi$)
    * if $\pi_g \neq \pi$, $\pi = \pi_g$ and go to .2
    * else $\pi$ is already optimal

All $\pi_g$ will be deterministic, the number of deterministic policies is finite, assuring the algorithm will eventually converge. Policy Iteration literally cuts through the search space which is quite powerful computationally.

### Value Iteration
The policy iteration presented above is quite flexible. and can be modified as needed depending on different scenarios. One way is ***Value Iteration***. Where We perform only one sweep (iteration) in policy evaluation using gradification. In other words, we just use:
$$ V(s) = max_a \sum_{a} \pi(a | s) \cdot \sum_{s'} \sum_{r} p(s', r | s, a) \cdot [r + \gamma \cdot v_{\pi}(s')]$$ 
* Avoiding full sweeps 
